# Konvertering fra korpus i pdf til et vasket korpus i ren tekst

## Etablering av et pdf-korpus

På [reiselivsforum.no](reiselivsforum.no) publiseres det avgjørelser fra Transportklagenemnda og Pakkereisenemnda.  Vi har lastet alle avgjørelsen i pdf-format. Nettsiden tilbyr ikke samlet nedlasting; det er nødvendig å lage et program som laster ned alle avgjørelser. Vi har utviklet et skript i Python modulen Scrapy som har lastet ned filer. Alle pdf-er er lastet ned til samme mappe. Pdf-ene er lagret med saksnummeret som filnavn. Søkemotoren på reiselivsforum.no oppgir totalt antall vedtak til 12853 pr. 15. oktober 2023. Skriptet lastet ned 12595 vedtak. Hva diskrepansen på 258 vedtak skyldes, vet vi ikke. Mange av de eldre vedtakene har et rettighetsforbehold fra Lovdata. Forbeholdet ser ut til å være en standardformulering og er den samme i alle vedtakene med forbehold:
> De publiserte nemndsavgjørelsene er bearbeidet av stiftelsen Lovdata, som innehar rettighetene til dokumentene. Utnyttelse av nemndsavgjørelsene til forlagsvirksomhet, distribusjon, drift av søkbare databaser eller opplæringsvirksomhet krever særskilt avtale med Lovdata. Lovdata avgjør i tvilstilfelle hva som faller inn under nevnte kategorier.

Her kan det være verdt å merke seg at forbeholdet står i vedtakene som er tilgjengelig for nedlasting nemndenes egen nettside. 

For å ikke komme i konflikt med rettighetsreservasjonen, fjerner vi de aktuelle vedtakene fra tekstkorpuset. Tilsammen fjerner vi da 4768 vedtak. De fordeler seg som følger.

| Nemnd                   | Vedtak med rettighetsforbehold |
|:------------------------| ------------------------------:|
|Pakkereiseklagenemnda    | 1734                           |
|Transportklagenemnda-Tog | 171                            |
|Transportklagenmnda-Fly  | 2863                           |
|**Sum:**                 |**4768**                        |

Vi står da igjen med 7827 vedtak fra 2017 til 2023. Søk i de gjenværende vedtakene bekrefter at de ikke har den samme rettighetsreservasjonen.

**Her må vi inn med en oversikt over antall vedtak pr. år. pr. nemnd. Kanskje det blir mulig å si noe om de 258 vi manglet også.**

## Konverting av pdf-er til ren tekst

For at avgjørelsene skal kunne brukes videre og behandles maskinelt, må pdf-ene konverteres til ren tekst. Nøyaktig konvertering til ren tekst er viktig for å kunne identifisere de ulike delene av vedtaket. Særlig kommer dette godt med når vi skal identifisere mindretallsvotum, og når vi skal eksperimentere med å bruke ulike deler av dokumentet som kontekst for språkmodellen (se under).

Vi prøver først å konvertere et vedtak i pdf til tekst med PyPDF2. 

In [32]:
import os
import PyPDF2

def pdf_to_text_pyPDF2(pdf_path):
    # Open the PDF file
    with open(pdf_path, 'rb') as file:
        # Create a PDF file reader
        pdf_reader = PyPDF2.PdfReader(file)
        
        # Loop through each page in the PDF file
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            # Extract text from each page
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    
    return text

# Specify the path to your PDF file
path_to_pdf = "./pdfs/2016-00003.pdf"
text = pdf_to_text_pyPDF2(path_to_pdf)
print(text)

Vedtak i Pakkereisenemnda
Sammendrag
Krav om dekning av merutgifter da togreisen "Norway in a nutshell" ble kansellert.
Dato
30.04.2017
Saksnummer
2016-00003
Tjenesteytere
Fjord Tours
Klager har i det vesentlige anført
Klager bestilte reisen "Norway in a nutshell" fra Oslo 15. - 16. oktober 2016 hos Fjord
Tours.
Først da klager ankom Myrdal, fant de ut at togreisen "Norway in a nutshell" var
kansellert. Klager ble ikke informert om dette.  
Hverken Fjord Tours eller NSB vil dekke klagers merutgifter til hotell, ﬂy, buss, taxi, tog og
tapt arbeidsfortjeneste.
Klager har fått refundert prisen for "Norway in a nutshell" med kr. 5.300, men krever å
få dekket de øvrige merutgiftene på kr. 6.270,57 fra Fjord Tours i og med at han bestilte
reisen hos Fjord Tours og ikke direkte hos NSB.
Det vises for øvrig til korrespondanse fra klager.
Tjenesteyterne har i det vesentlige anført
Fjord Tours har anført at saken er avsluttet fra Fjord Tours side da de i mail av 18.10.2016
informerte om at de vi

Som vi ser mangler det en del struktur. Det er vanskelig å identifisere overskrifter og avsnitt. Vi prøver derfor å konvertere pdf-ene til ren tekst med **tika**. Tika returner tekst med bedre struktur, men den kan forbedres litt:
- Vi får en del blanke linjer i begynnelsen og slutten av vedtakene. Dette kan vi fjerne med strip-funksjonen.
- Vi får noen ekstra linjeskift der pdf-ene har sideskift. 

Innspeksjon av flere vedtak viser at sideskift i pdf-en konverteres til fire linjeskift, dvs. `("\n\n\n\n")`. Hvis vi fjerner alle fire vil vi få sammenhengende tekst, men i noen tilfeller skal det være linjeskift på samme sted som sideskiftet. Innspeksjon av konverterte tekstfiler viser at det ofte er et mellomrom foran de fire linjeskiftene der det skal være et linjeskift, men dette gjelder ikke alltid. Vi velger derfor å fjerne to linjeskift som et utgangspunkt. Da unngår vi å fjerne linjeskift der det skal være et. I tilfeller der linjen etter sideskiftet begynner med liten bokstav antar vi at det bare skal være et ordinært linjeskift. Da fjerner vi tre linjeskift. Denne tilnærmingen løser mye, men har en svakhet: Vi skiller ikke mellom sideskift som kommer der det er nytt avsnitt i teksten og sideskift der det kommer en ny setning i teksten. Konsekvensen er at sideskift som faller sammen med at ny setning i samme avsnitt påbegynnes, innfører en ekstra avsnittsmarkering. For vårt formål virker dette å være uproblematisk.

In [46]:
from tika import parser
import re

parsed_pdf = parser.from_file("./pdfs/2016-00003.pdf")
vedtak_txt = parsed_pdf['content'].strip()

# Bruker et regulært uttrykk for å identifisere sideskift etterfulgt av liten forbokstav
pattern = r'\n\n\n\n(?=[a-z])'
vedtak_txt = re.sub(pattern, '\n', vedtak_txt)

# Resten av sideskiftene bytter vi ut med to linjeskift
vedtak_txt = vedtak_txt.replace('\n\n\n\n', '\n\n')
# print(repr(vedtak_txt)) # repr() viser linjeskift i teksten
print(vedtak_txt)


Vedtak i Pakkereisenemnda

Sammendrag

Krav om dekning av merutgifter da togreisen "Norway in a nutshell" ble kansellert.

Dato
30.04.2017

Saksnummer
2016-00003

Tjenesteytere
Fjord Tours

Klager har i det vesentlige anført

Klager bestilte reisen "Norway in a nutshell" fra Oslo 15. - 16. oktober 2016 hos Fjord
Tours.

Først da klager ankom Myrdal, fant de ut at togreisen "Norway in a nutshell" var
kansellert. Klager ble ikke informert om dette.  

Hverken Fjord Tours eller NSB vil dekke klagers merutgifter til hotell, fly, buss, taxi, tog og
tapt arbeidsfortjeneste.

Klager har fått refundert prisen for "Norway in a nutshell" med kr. 5.300, men krever å
få dekket de øvrige merutgiftene på kr. 6.270,57 fra Fjord Tours i og med at han bestilte
reisen hos Fjord Tours og ikke direkte hos NSB.

Det vises for øvrig til korrespondanse fra klager.

Tjenesteyterne har i det vesentlige anført
Fjord Tours har anført at saken er avsluttet fra Fjord Tours side da de i mail av 18.10.2016
informert

Resultatet er bedre. Mye struktur er bevart, og vi har fjernet en rekke unødvendige linjeskift.

*Arbeidet med å få orden på teksten med linjeskift og avsnitt burde i og for seg ha vært unødvendig. I Lovdata ligger teksten ferdig kodet med riktig struktur, men den er altså ikke allment tilgjengelig.* 

## Konvertere alle vedtak til tekst

Etter denne innledende utprøvingen, konverterer utvider vi koden slik at vi konverterer alle vedtak til ren tekst.
(Vi skal gjøre en operasjon til på vedtakene, men skiller denne ut til et eget etterfølgende steg)

In [48]:
from tika import parser
import os
import re

pdf_directory = './pdfs/'
txt_directory = './txt/'

for filename in os.listdir(pdf_directory):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_directory, filename)
        parsed_pdf = parser.from_file(pdf_path)
        vedtak_txt = parsed_pdf['content'].strip()

        # Bruker et regulært uttrykk for å identifisere sideskift etterfulgt av liten forbokstav
        pattern = r'\n\n\n\n(?=[a-z])'
        vedtak_txt = re.sub(pattern, '\n', vedtak_txt)

        # Resten av sideskiftene bytter vi ut med to linjeskift
        vedtak_txt = vedtak_txt.replace('\n\n\n\n', '\n\n')
        
        # Lagrer vedtak som ren tekst.
        
        # Setter sammen nytt navn til filene
        txt_filename = os.path.splitext(filename)[0] + '.txt'  
        txt_path = os.path.join(txt_directory, txt_filename)
        
        # Sjekker om folderen for lagring finnes, hvis ikke oppretter vi en
        if not os.path.exists(txt_directory):
            os.makedirs(txt_directory)

        # Lagrer filen
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(vedtak_txt)


        
        
        

Resultatet er 7827 vedtak konvert til ren tekst og lagret som individuelle filer med endelsen .txt.

## Fjerne nemndas representanter fra vedtaksteksten

Alle vedtak ender med en liste over nemndas representanter. Til vårt formål er det ikke nødvendig å ha med denne informasjonen videre. Vi skal derfor slette denne informasjonen får vi tar i bruk vedtakene.

For å være sikre på at vi bare sletter nemndas representanter, kjører vi en test. Vi identifiserer da overskriften "Nemndas representanter" og sjekker hvor mange linjer tektst denne delen består av. 

Først sjekker vi om alle vedtakene har en slik overskrift.



In [57]:
import os

txt_directory = './txt/'
search_string = "Nemndas representanter\n"
teller = 0 # variabel for å telle hvor mange filer vi har sjekket

# Looper over alle filer i folderen
for filename in os.listdir(txt_directory):
    if filename.endswith('.txt'):  
        txt_path = os.path.join(txt_directory, filename)  
        
        # Leser inn filen
        with open(txt_path, 'r', encoding='utf-8') as txt_file:  
            file_content = txt_file.read()  
            teller += 1
            # Sjekker om det søkestrengen ikke er i filen
            if search_string not in file_content:
                print(f'"{filename}" inneholder ikke teksten "{search_string.strip()}"')
       #    Kode for å sjekke at filene faktisk inneholder strengen.
       #     if search_string  in file_content:
       #         print(f'"{filename}" inneholder teksten "{search_string.strip()}"')
                

print(teller)


7827


Før vi fjerner teksten sjekker vi hvor mange linjer som kommer etter "Nemndas representanter"

In [65]:
import os
import statistics  
txt_directory = './txt/'
search_string = "Nemndas representanter"
resultater = []  # liste for å lagre resultatene som tupler (filnavn, antall_linjer, antall_tegn)

# Looper over alle filer i mappen
for filename in os.listdir(txt_directory):
    if filename.endswith('.txt'):
        txt_path = os.path.join(txt_directory, filename)

        # Leser filen
        with open(txt_path, 'r', encoding='utf-8') as txt_file:
            file_content = txt_file.read()

            # Finn posisjonen til søkestrengen i filinnholdet
            index = file_content.find(search_string)

            # Hvis strengen ble funnet, beregne linjer og tegn etter den
            if index != -1:
                # Legg til lengden på søkestrengen til indeksen for å starte tellingen etter strengen
                index += len(search_string)

                # Hent innholdet etter søkestrengen
                remaining_content = file_content[index:]

                # Tell antall linjer ved å splitte det gjenværende innholdet med nye linjer
                line_count = remaining_content.count('\n')

                # Tell antall tegn i det gjenværende innholdet
                character_count = len(remaining_content)

                # Lagre resultatene i listen som en tuple
                resultater.append((filename, line_count, character_count))

# Beregn og skriv ut beskrivende statistikk
if results:  # sjekk om resultatlisten ikke er tom
    # Pakk ut antall linjer og tegn
    _, line_counts, char_counts = zip(*results)

    # Beregning av statistikk for antall linjer
    max_lines = max(line_counts)
    min_lines = min(line_counts)
    avg_lines = statistics.mean(line_counts) 

    # Beregning av statistikk for antall tegn
    max_chars = max(char_counts)
    min_chars = min(char_counts)
    avg_chars = statistics.mean(char_counts) 

    # Skriver ut statistikken
    print(f"Maks linjer: {max_lines}, Min linjer: {min_lines}, Gjennomsnitt antall linjer: {avg_lines:.2f}")
    print(f"Maks tegn: {max_chars}, Min tegn: {min_chars}, Gjennomsnitt antall tegn: {avg_chars:.2f}")
else:
    print("Ingen resultater funnet.")

print(f'Totalt antall sjekkede filer: {len(results)}')


Maks linjer: 8, Min linjer: 2, Gjennomsnitt antall linjer: 6.02
Maks tegn: 198, Min tegn: 9, Gjennomsnitt antall tegn: 151.30
Totalt antall sjekkede filer: 7827


Resultatet er betryggende. Vedtak med kun to linjer er «tomme» vedtak, der resultatet er et forlik mellom partene. Vi fjerner derfor overskriften "Nemndas vedtak" og resten av teksten som følger etter.

In [66]:
import os

txt_directory = './txt/'
search_string = "Nemndas representanter\n"  # Ny linje-tegnet er lagt til for å sikre at vi finner den eksakte linjen

# Looper over alle filer i mappen
for filename in os.listdir(txt_directory):
    if filename.endswith('.txt'):
        txt_path = os.path.join(txt_directory, filename)

        # Leser filen
        with open(txt_path, 'r', encoding='utf-8') as txt_file:
            file_content = txt_file.readlines()  # readlines() brukes for å få en liste av linjer

        # Sjekker om søkestrengen er i filinnholdet
        line_found = False
        for line in file_content:
            if search_string in line:
                line_found = True
                break

        # Hvis linjen ble funnet, fjern den og innholdet som følger
        if line_found:
            # Få indeksen til linjen som inneholder søkestrengen
            index = file_content.index(search_string)
            
            # Fjern funnet linje og alt etter det
            file_content = file_content[:index]

            # Lagre det modifiserte innholdet tilbake til den samme filen
            with open(txt_path, 'w', encoding='utf-8') as txt_file:
                txt_file.writelines(file_content)


Resultatet er en folder med alle vedtak i tekstformat. Alle vedtak er renset for informasjon om nemndas representanter. 